In [1]:
%matplotlib widget

import tensorflow as tf
import os
import sys
import cv2
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import random

from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import GlobalMaxPooling2D
from numpy.linalg import norm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [2]:
DATASET_PATH = "/Users/jeremy/Google Drive/datasets/fashion-dataset/"
print(os.listdir(DATASET_PATH))

['embeddings.tsv', '.DS_Store', 'images.csv', 'images', 'styles', 'styles.csv', '.ipynb_checkpoints', 'embeddings.csv', 'resnet50-embeddings.pkl']


In [3]:
df = pd.read_csv(DATASET_PATH + "styles.csv", nrows=5000, error_bad_lines=False)
df['image'] = df.apply(lambda row: str(row['id']) + ".jpg", axis=1)
df = df.reset_index(drop=True)
df.head(10)

,id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,image
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,15970.jpg
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,39386.jpg
2,59263,Women,Accessories,Watches,Watches,Silver,Winter,2016,Casual,Titan Women Silver Watch,59263.jpg
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,21379.jpg
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,53759.jpg
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,1855.jpg
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,30805.jpg
7,26960,Women,Apparel,Topwear,Shirts,Purple,Summer,2012,Casual,Jealous 21 Women Purple Shirt,26960.jpg
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012,Casual,Puma Men Pack of 3 Socks,29114.jpg
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,30039.jpg


In [4]:
def get_img_path(img):
  return DATASET_PATH + "images/" + img

In [5]:
df_embs = pickle.load(open(DATASET_PATH + "resnet50-embeddings.pkl", "rb"))

In [6]:
# Filter only embeddings with master category 'Apparel' and save indices to filter for them later
df_filtered = df.loc[df.masterCategory == 'Apparel']
df_filtered_idx = df_filtered.index.values.tolist()
df_filtered = df_filtered.reset_index(drop=True)
df_filtered.masterCategory.unique(), df_filtered.shape, len(df_filtered_idx)

(array(['Apparel'], dtype=object), (2297, 11), 2297)

In [7]:
embs_filtered = []
for i, row in df_embs.iterrows():
    if i in  df_filtered_idx:
        embs_filtered.append(row)
df_embs_filtered = pd.DataFrame(embs_filtered)
df_embs_filtered = df_embs_filtered.reset_index(drop=True)
df_embs_filtered

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,5.661316,2.366165,0.000000,3.255730,0.845208,2.673841,8.780870,6.280786,0.787188,1.587657,...,2.463610,0.000000,1.825994,20.485268,3.902863,0.000000,8.225969,1.313863,0.000000,10.820526
1,5.790417,9.593648,0.000000,9.431670,0.287235,0.259945,17.028748,3.037453,0.136341,0.000000,...,1.670570,0.013891,10.338681,4.784722,0.189140,0.000000,7.739144,0.372693,0.000000,8.903802
2,5.882203,1.958606,0.000000,12.519882,0.000000,0.000000,9.383594,3.781614,0.000000,1.490617,...,17.825478,0.000000,3.592507,2.042305,0.000000,0.000000,6.656779,1.457314,1.694204,1.403327
3,0.245968,17.383430,0.493824,3.374468,2.567368,0.000000,6.482537,5.862639,0.000000,3.164866,...,4.464024,0.447075,1.448143,12.884138,0.000000,0.397501,5.028075,1.939697,0.000000,11.485400
4,0.028546,18.567099,0.000000,0.757250,1.504949,0.000000,1.982024,2.963777,1.072765,4.539300,...,3.247082,4.273134,3.088643,16.997053,0.000000,0.000000,2.258095,1.595798,0.000000,7.674047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,3.134137,12.337814,3.281830,0.000000,6.123401,6.273666,3.345068,4.230134,2.082114,2.691854,...,5.392367,2.139013,7.540308,5.394151,6.070920,0.000000,0.000000,0.000000,0.000000,10.168349
2293,2.800386,19.866081,3.215006,2.487933,0.147357,0.000000,7.958175,5.669247,0.706661,4.877097,...,10.292748,0.000000,0.738173,16.109261,0.383140,0.000000,4.432935,2.898650,0.416252,12.830476
2294,4.801172,16.536064,2.132150,2.630906,1.190370,0.243107,1.454644,3.500423,0.627453,5.221931,...,0.000000,1.886256,3.294083,17.186272,0.000000,2.014549,1.663332,2.118907,0.000000,6.815829
2295,3.184294,14.538852,6.306130,0.000000,3.361131,6.701039,0.729837,0.295319,0.252367,3.175583,...,0.000000,1.471544,1.482233,22.654318,2.425646,12.427614,8.393599,0.000000,9.616654,3.375090


In [8]:
# Perform PCA over the embeddings to reduce dimensionality
num_feature_dimensions = 2000  # Set the number of embedding dimensions
pca = PCA(n_components = num_feature_dimensions)
embs_compressed = pca.fit_transform(df_embs_filtered)
df_embs_filtered_compressed = pd.DataFrame(embs_compressed)
df_embs_filtered_compressed

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,-64.281487,-0.382144,10.129684,-22.772327,-38.308056,10.126407,36.410957,-46.646206,-34.147419,8.802602,...,-0.072885,0.021689,0.262467,-0.307515,0.193920,0.094781,0.169586,0.072440,-0.110927,-0.087643
1,-50.859097,-78.791565,-48.603271,-5.542880,57.489540,-30.835356,-2.337835,-15.355940,-7.670216,-7.935432,...,0.285969,-0.216813,0.071849,0.087859,0.111866,0.068268,-0.092981,-0.335568,0.014027,-0.111936
2,-21.894207,-69.084801,-56.701370,-13.024130,48.613529,-35.150982,6.533363,12.986199,1.046159,-23.647963,...,-0.028149,0.022253,0.154494,0.156459,-0.252553,-0.105450,0.171465,-0.026870,-0.177849,-0.224921
3,-52.814507,-30.695364,24.538889,18.532671,-15.459037,-4.578600,-20.258459,5.052645,-18.399387,14.211772,...,0.025601,0.105441,-0.200961,0.347842,-0.301099,0.115235,0.168436,-0.020016,0.313665,-0.051148
4,-26.194756,-12.308571,56.018593,28.274645,4.040835,0.309532,20.197136,11.746011,-15.117881,0.662400,...,0.197897,-0.032425,-0.240887,0.149448,-0.014207,0.179669,-0.008225,-0.106300,-0.285183,-0.078262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292,99.575592,-30.713884,-20.068476,-25.830282,-22.531815,51.434803,-40.078777,-24.682608,-38.468166,53.920856,...,0.060048,0.044371,0.052964,-0.066104,-0.138551,-0.001413,0.052767,0.004295,-0.079621,-0.033358
2293,-26.236425,-12.102105,52.941227,21.009844,-2.619762,-23.604671,3.913752,4.513397,-3.499625,-43.001522,...,-0.087290,0.230910,0.315426,-0.051655,0.143980,-0.011697,-0.294969,0.034188,-0.218997,0.069627
2294,-29.318754,36.647430,31.197569,14.334860,26.817410,-5.634321,-21.891491,-2.816172,-21.250452,2.923827,...,-0.164836,0.186601,0.309618,0.173436,0.280081,-0.082836,0.100793,0.167939,-0.015266,0.083564
2295,54.131542,63.903694,-8.761940,-63.546387,17.439548,9.299923,16.524239,42.935120,53.401096,43.313850,...,-0.091945,-0.053217,0.037641,0.147787,0.061091,0.060630,0.099054,-0.040325,-0.055460,-0.106942


In [9]:
# Map season values to either Spring or Winter
for i, row in df_filtered.iterrows():
    if df_filtered.at[i, 'season'] == "Spring":
        df_filtered.at[i, 'season'] = "Summer"
    elif df_filtered.at[i, 'season'] == "Fall":
        df_filtered.at[i, 'season'] = "Winter"
df_filtered.season.unique()

array(['Winter', 'Summer'], dtype=object)

In [10]:
# Application of SVM to create the axis for a given feature
X = df_embs_filtered_compressed
y = df_filtered["season"]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
svm_clf = LinearSVC(C=1, max_iter=100000)
svm_clf.fit(X_scaled, y)

LinearSVC(C=1, max_iter=100000)

In [11]:
def get_nearest_neighbour(user_position, df, embs):
   nearest_neighbour = None
   smallest_dist = sys.maxsize
   for row in df.itertuples():
      embedding_position = embs.iloc[row.Index].to_numpy()
      dist = norm(user_position - embedding_position)
      if dist < smallest_dist:
         smallest_dist = dist
         nearest_neighbour = row.id
   return nearest_neighbour

In [12]:
def annotate_nearest_neighbour(nearest_neighbour, df):
    arr_img = plt.imread(get_img_path(df.loc[df['id'] == nearest_neighbour].image.values[0]))
    plt.imshow(arr_img)

In [13]:
def compute_navigation_axis(emb, w, range):
    nav_axis = []
    for i, feature in enumerate(emb):
        feature_val = emb[feature].values[0]
        if w[i] > 0:
            pos_range = np.arange(start=feature_val, stop=feature_val + range, step=w[i])
            neg_range = np.arange(start=feature_val, stop=feature_val - range, step=-w[i])
            nav_axis.append(np.sort(np.concatenate((pos_range, neg_range))))
        else:
            pos_range = np.arange(start=feature_val, stop=feature_val + range, step=-w[i])
            neg_range = np.arange(start=feature_val, stop=feature_val - range, step=w[i])
            nav_axis.append(np.sort(np.concatenate((pos_range, neg_range))))
    return nav_axis

In [14]:
def initialize_user_position(axis, idx):
    user_position = []
    for dim in axis:
        user_position.append(dim[idx])
    return user_position

In [15]:
def update_user_position(change):
    """Update the user position after the slider value has changed"""
    idx = np.where(np.isclose(nav_axis[0], min(nav_axis[0], key=lambda x:abs(x-change.new))))[0][0]
    new_user_pos = []
    for i in range(len(nav_axis)):
        new_user_pos.append(nav_axis[i][idx])
    user_position = new_user_pos
    nearest_neighbour = get_nearest_neighbour(user_position, df_filtered, df_embs_filtered_compressed)
    annotate_nearest_neighbour(nearest_neighbour, df_filtered)
    plt.title('Nearest Embedding: {} with season: {}'.format(nearest_neighbour, df_filtered.loc[df_filtered['id'] == nearest_neighbour].season.values[0]))
    fig.canvas.draw()
    fig.canvas.flush_events()

In [18]:
min_axis_len = 100000
for i in range(len(nav_axis)):
    if len(nav_axis[i]) < min_axis_len:
        min_axis_len = len(nav_axis[i])
min_axis_len

1116

In [33]:
from ipywidgets import AppLayout, FloatSlider

# Create matplotlib figure for displaying fashion items
fig = plt.figure(figsize=(7, 7))

# Create orthogonal navigation axis
w = svm_clf.coef_[0]
rand_emb = df_embs_filtered_compressed.sample()
nav_axis = compute_navigation_axis(rand_emb, w, 100)

min_axis_len = 100000
for i in range(len(nav_axis)):
    if len(nav_axis[i]) < min_axis_len:
        min_axis_len = len(nav_axis[i])
min_axis_len

starting_idx = np.where(np.isclose(nav_axis[0], min(nav_axis[0], key=lambda x:abs(x-rand_emb[0].values[0]))))[0][0]
user_position = initialize_user_position(nav_axis, starting_idx)

nearest_neighbour = get_nearest_neighbour(user_position, df_filtered, df_embs_filtered_compressed)
annotate_nearest_neighbour(nearest_neighbour, df_filtered)

plt.title('Nearest Embedding: {} with season: {}'.format(nearest_neighbour, df_filtered.loc[df_filtered['id'] == nearest_neighbour].season.values[0]))

# Create Slider to navigate in embedding space
slider = FloatSlider(
    orientation="horizontal",
    description="x-Position:",
    value=user_position[0],
    min=min(nav_axis[0][:min_axis_len]),
    max=max(nav_axis[0][:min_axis_len])
)
slider.layout.margin = '0px 10% 0px 10%'
slider.layout.width = '40%'

slider.observe(update_user_position, names='value')

display(slider)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

FloatSlider(value=-15.002667427062988, description='x-Position:', layout=Layout(margin='0px 10% 0px 10%', widt…